Note: use `yapf -i name.py` to formulate the python code

For faster compilation, use `ti.core.toggle_advanced_optimization(false)`

# Lagrangian view versus Eular's view

Lagrangian view: sensors that move passively with the simulated material. 

Eular's view: Sensors are still and never move

<img src="/Users/yutingzhang/Desktop/softness_perception/IMG_0316.PNG">

## Mass-spring systems

Hooke's law 
$f_ij = -k(\left\lVert \mathbf{X_i} - \mathbf{X_j} \right\rVert_2 - l_ij)(\hat{\mathbf{X_i}-\mathbf{X_j}})$
(Note: the stiffness coefficient times the change between stretched and rest spring times the direction vector from particle i to particle j. the force experienced by one point in space)

$\mathbf{f}_i = \sum \limits_{j}^{j\neq i} \mathbf{f}_{ij}$
(Note: sum them all up. The sum of forces experienced by one point)

Newton's second law of motion
$\frac{\partial \mathbf{V}_i}{\partial t} = \frac{1}{m_i}\mathbf{f}_i$

$\frac{\partial \mathbf{X}_i}{\partial t} = \mathbf{v}_i$

## Time integration

1. Forward Euler (explicit)

$\mathbf{v}_{t+1} = \mathbf{v}_t + \delta t \frac{\mathbf{f}_t}{m}$

$\mathbf{x}_{t+1} = \mathbf{x}_t + \delta t \mathbf{v}_t$

2. Semi-implicit Euler, symplectic Euler, explicit

$\mathbf{v}_{t+1} = \mathbf{v}_t + \delta t \frac{\mathbf{f}_t}{m}$

$\mathbf{x}_{t+1} = \mathbf{x}_t + \delta t \mathbf{v}_{t+1}$

## implementation
Compute new velocity using $\mathbf{v}_{t+1} = \mathbf{v}_t + \delta t \frac{\mathbf{f}_t}{m}$

Collision with the ground

Compute the new position using $\mathbf{x}_{t+1} = \mathbf{xj}_t + \delta t \mathbf{v}_{t+1}$

#### why do we need the collision with the ground first before computing new position? 
Need to know whether the point has already gone underground or not. If so, you need to set the velocity along the y-dimension to 0. 


## Explicit vs implicit time integrators
For explicit time integrator, future depends only on the past.

For implicit time integrator, future depends on both future and past. Each time step is more expensive but time steps are larger. 

### Mass-spring systems, implicit time integration
$\mathbf{x}_{t+1} = \mathbf{x}_t + \delta t\mathbf{v}_{t+1}$

$\mathbf{v}_{t+1} = \mathbf{v}_t + \delta t\mathbf{M}^{-1} \mathbf{f}(\mathbf{x}_{t+1})$

$\mathbf{v}_{t+1} = \mathbf{v}_t + \delta t\mathbf{M}^{-1} \mathbf{f}(\mathbf{x}_t + \delta t\mathbf{v}_{t+1})$

After linearization: $\mathbf{v}_{t+1} = \mathbf{v}_t + \delta t\mathbf{M}^{-1} [\mathbf{f}(\mathbf{x}_t) + \frac{\delta \mathbf{f}}{\delta \mathbf{x}}(\mathbf{x}_t) \delta{t}\mathbf{v}_{t+1}]$

After cleaning up: $[\mathbf{I} - \delta t^{2}\mathbf{M}^{-1}\frac{\delta \mathbf{f}}{\delta \mathbf{x}}(\mathbf{x}_t)] \mathbf{v}_{t+1} = \mathbf{v}_t + \delta t \mathbf{M}^{-1}\mathbf{f}(\mathbf{x}_t)$

##### Combining everything together, we get an unifying explicit and implict integrators
$[\mathbf{I} - \beta \delta t^{2}\mathbf{M}^{-1}\frac{\delta \mathbf{f}}{\delta \mathbf{x}}(\mathbf{x}_t)] \mathbf{v}_{t+1} = \mathbf{v}_t + \delta t \mathbf{M}^{-1}\mathbf{f}(\mathbf{x}_t)$

when $\beta = 0$, forward/semi implicit Euler. 

when $\beta = 0.5$, middle-point. 

when $\beta = 1$, backward Euler. 


## Smooth particle hydrodynamics
We use particles carrying samples of physical quantities and a kernel function W to approximate continuous fields. It's more like a weighted equation. 

$\mathbf{A}(\mathbf{x}) = \sum\limits_{i}\mathbf{A}_i\frac{m_i}{\mathbf{\rho}_i}\mathbf{W}(\left\lVert \mathbf{X} - \mathbf{X_j} \right\rVert_2, h)$

Usually people use weakly compressible SPH to approximate SPH

Momentum equation:

$\frac{D\mathbf{v}}{Dt} = -\frac{1}{\mathbf{\rho}}\nabla p + \mathbf{g}$
(Note: D represents the material derivative, gradient of pressue divided by the density represents accelaration. The pressure is calculated as $p = B((\frac{\rho}{\rho_p})^\gamma)-1$, which suggests that the higher the density is, the higher the pressure is. $B$ is bulk modulus)

$\mathbf{A}(\mathbf{x}) = \sum\limits_{i}\mathbf{A}_i\frac{m_i}{\mathbf{\rho}_i}\mathbf{W}(\left\lVert \mathbf{X} - \mathbf{X_j} \right\rVert_2, h)$ and $\mathbf{\rho}_i = \sum\limits_{i}m_j\mathbf{W}(\left\lVert \mathbf{X_i} - \mathbf{X_j} \right\rVert_2,h)$. $\rho$ equation suggests that the density of each mass point is calculated using weighted mass. 

## SPH simulation cycle
$\frac{D\mathbf{v}}{Dt} = -\frac{1}{\mathbf{\rho}}\nabla p + \mathbf{g}$

$p = B((\frac{\rho}{\rho_p})^\gamma)-1$

1. For each particle $i$, compute $\mathbf{\rho}_i = \sum\limits_{i}m_j\mathbf{W}(\left\lVert \mathbf{X_i} - \mathbf{X_j} \right\rVert_2,h)$. Each particle doesn't carry density, but each particle carries a mass. The density is computed using the weighted summary of mass. 

2. For each particle $i$, compute $\nabla p_i$ using the gradient operator. 
3. Symplectic Euler step:

$\mathbf{v}_{t+1} = \mathbf{v}_{t} + \delta t \frac{D_\mathbf{v}}{D_t}$

$\mathbf{x}_{t+1} = \mathbf{x}_t + \delta t \mathbf{v}_{t+1}$

## Accelerating SPH: Neighborhood search
Per substep complexity of SPH is $O(n^2)$. 
After building a spatial data structure such as voxel grids to keep track of the particles within each spatial location, we can easily search the neighbors of specific voxels. This reduces the complexity to $O(n)$. For instance, we only need to search for the 3*3 grid around the voxel. 

## Deformation
Deformation map $\phi$: a (vector to vector) function that relates rest material position and deformed material position. 

$\mathbf X_{deformed} = \phi(\mathbf X_{rest})$

Deformation gradient F is defined as $\frac{\delta\mathbf X_{deformed}}{\delta\mathbf X_{rest}}$

The deformation gradient is the same regardless of the constant added at the end of the $\mathbf X_{deformed} = \phi(\mathbf X_{rest})$

Deform/rest volume ratio $J = det(\mathbf F)$

## Hyperelasticity
What is elasticity? Elasticity is the ability of return back to the original shape. 

Hyperelastic materials: materials whose stress-strain relationship is defined by a strain energy density function: $\mathbf\psi = \mathbf\psi(\mathbf F)$

Intuitive understanding: $\psi$ is a potential function that penalizes deformation. 

"Stress": the material's internal elastic forces. 

"Strain": just replace it with deformation gradient $\mathbf F$. 

## Stress tensor
Stress stands for internal forces that infinitesimal material components exert on their neighborhood. 

Based on our need, we use different measures of stress. 
- The first Piola-Kirchhoff stress tensor (PK1): $\mathbf P(F) = \frac{\delta\psi(\mathbf F)}{\delta\mathbf F}$(Note: this is in the rest space. ) 
- Kirchhoff stress $\tau$ 
- Cauchy stress tensor: $\sigma$ (Note: Symmetric, because of conservation of angular momentum)

Relationship: $\tau = J\sigma = \mathbf PF^T$,  $\mathbf P = J\sigma\mathbf F^{-T}$,  Traction $t = \sigma^Tn$

## Elastic moduli (isotropic materials, materials that remain the same when tested in different directions.)
Coefficients to quantify elasticity. 

Young's modulus $E = \frac{\sigma}{\epsilon}$

Bulk modulus $K = -\mathbf V\frac{dP}{dV}$

Poisson's ratio $\mathbf v\in [0.0,0.5)$


Lamé parameters: 

Lamé first parameter $\mu$

Lamé second parameter $\sigma$ (aka. shear modulus, denoted by $G$)

Useful conversion between formula: 

$K = \frac{E}{3(1-2v)}$,  $\sigma = \frac{Ev}{(1+v)(1-2v)}$, $\mu = \frac{E}{2(1+v)}$

## The finite element method
Galerkin discretization scheme that builds discrete equations using weak formulations of continuous PDEs. 